<a href="https://colab.research.google.com/github/sclaudiobr/covid2019/blob/master/COVID19_Heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade folium

     |████████████████████████████████| 102kB 2.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [3]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
folium.__version__

'0.11.0'

In [0]:
df = pd.read_csv('https://brasil.io/dataset/covid19/caso?format=csv')

In [5]:
df.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-05-07,AC,Acrelândia,city,15,1,True,15256.0,1200013.0,98.32197,0.0667
1,2020-05-07,AC,Assis Brasil,city,1,0,True,7417.0,1200054.0,13.48254,0.0000
2,2020-05-07,AC,Bujari,city,4,0,True,10266.0,1200138.0,38.96357,0.0000
3,2020-05-07,AC,Cruzeiro do Sul,city,26,0,True,88376.0,1200203.0,29.41975,0.0000
4,2020-05-07,AC,Epitaciolândia,city,2,0,True,18411.0,1200252.0,10.86307,0.0000


In [0]:
cidades = pd.read_csv('https://raw.githubusercontent.com/sandeco/CanalSandeco/master/covid-19/cidades_brasil.csv')

In [7]:
cidades.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [0]:
# trocando o index

cidades = cidades.set_index('codigo_ibge')

In [9]:
cidades.head()

,nome,latitude,longitude,capital,codigo_uf
codigo_ibge,,,,,
5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
5200100,Abadiânia,-16.19700,-48.7057,0,52
3100203,Abaeté,-19.15510,-45.4444,0,31
1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [10]:
cities = df.loc[df.place_type == 'city',:]
cities.place_type.unique()

array(['city'], dtype=object)

In [11]:
len(cities)

57271

In [12]:
# juntar as duas bases de dados (IBGE cidades) com cities
cities = cities.join(cidades, on='city_ibge_code')
cities.head(5)

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,nome,latitude,longitude,capital,codigo_uf
0,2020-05-07,AC,Acrelândia,city,15,1,True,15256.0,1200013.0,98.32197,0.0667,Acrelândia,-9.82581,-66.8972,0.0,12.0
1,2020-05-07,AC,Assis Brasil,city,1,0,True,7417.0,1200054.0,13.48254,0.0000,Assis Brasil,-10.92980,-69.5738,0.0,12.0
2,2020-05-07,AC,Bujari,city,4,0,True,10266.0,1200138.0,38.96357,0.0000,Bujari,-9.81528,-67.9550,0.0,12.0
3,2020-05-07,AC,Cruzeiro do Sul,city,26,0,True,88376.0,1200203.0,29.41975,0.0000,Cruzeiro do Sul,-7.62762,-72.6756,0.0,12.0
4,2020-05-07,AC,Epitaciolândia,city,2,0,True,18411.0,1200252.0,10.86307,0.0000,Epitaciolândia,-11.01880,-68.7341,0.0,12.0


In [13]:
# selecionar as colunas de interesse
geo_last = cities.loc[cities.is_last==True,['city','latitude', 'longitude','state','confirmed','deaths']]
geo_last.head()

,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,15,1
1,Assis Brasil,-10.92980,-69.5738,AC,1,0
2,Bujari,-9.81528,-67.9550,AC,4,0
3,Cruzeiro do Sul,-7.62762,-72.6756,AC,26,0
4,Epitaciolândia,-11.01880,-68.7341,AC,2,0


In [14]:
#Quantidade de cidades
len(geo_last)

2664

In [15]:
#Quantidade de casos confirmados
geo_last.confirmed.sum()

134667

In [16]:
#Quantidade de mortes
geo_last.deaths.sum()

9166

In [17]:
#Porcentagem de mortos sobre os casos confirmados
geo_last.deaths.sum()/geo_last.confirmed.sum()*100

6.806418795993078

In [18]:
# verificar se todos os estados estão inseridos
len(geo_last.state.unique())

27

In [19]:
coordenadas = geo_last[['latitude','longitude','confirmed']]
#retirar as linhas que tiver dados faltando
coordenadas = coordenadas.dropna()
coordenadas.head()

,latitude,longitude,confirmed
0,-9.82581,-66.8972,15
1,-10.92980,-69.5738,1
2,-9.81528,-67.9550,4
3,-7.62762,-72.6756,26
4,-11.01880,-68.7341,2


In [0]:
baseMap = folium.Map(
    width = '100%',   height = '100%',
    location = [-15.77972, -47.92972], # <- Brasil
    zoom_start = 4
)

In [21]:
baseMap

In [0]:
baseMap = baseMap.add_child(plugins.HeatMap(coordenadas))

In [23]:
baseMap

In [24]:
# retirar dados faltantes
geo_last = geo_last.dropna()
geo_last.head(5)

,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,15,1
1,Assis Brasil,-10.92980,-69.5738,AC,1,0
2,Bujari,-9.81528,-67.9550,AC,4,0
3,Cruzeiro do Sul,-7.62762,-72.6756,AC,26,0
4,Epitaciolândia,-11.01880,-68.7341,AC,2,0


In [25]:
geo_last.iloc[1]['latitude']

-10.9298

In [0]:
for i in range(0,len(geo_last)):
    folium.Circle(
        location = [geo_last.iloc[i]['latitude'], geo_last.iloc[i]['longitude']],
        color = '#00FF69',
        fill = '#00A1B3',
        tooltip = '<li><bold> CIDADE: '+ str(geo_last.iloc[i]['city']) + "</li></bold>" +
                 '<li><bold> ESTADO: '+ str(geo_last.iloc[i]['state']) +"</li></bold>" +
                 '<li><bold> CONFIRMADOS: '+ str(geo_last.iloc[i]['confirmed'])+"</li></bold>" +
                 '<li><bold> MORTES: '+ str(geo_last.iloc[i]['deaths']) + "</li></bold>" ,
        radius = (geo_last.iloc[i]['confirmed']**1.1)
                                            
    ).add_to(baseMap)

In [27]:
# importando logomarca e colocando no mapa
from folium.plugins import FloatImage
url = 'https://raw.githubusercontent.com/sclaudiobr/algoritmos-py/master/text43.png'
FloatImage(url,bottom=10,left=55).add_to(baseMap)
baseMap

In [0]:
baseMap.save('heatmap-cidade.html')